# Cluster analysis

In this notebook, we will look at how to debug blocked table similarities and the resulting clusters.

In [16]:
from dask.distributed import Client
Client?

Init signature:
Client(
    address=None,
    loop=None,
    timeout='__no_default__',
    set_as_default=True,
    scheduler_file=None,
    security=None,
    asynchronous=False,
    name=None,
    heartbeat_interval=None,
    serializers=None,
    deserializers=None,
    extensions=[<class 'distributed.pubsub.PubSubClientExtension'>],
    direct_to_workers=None,
    connection_limit=512,
    **kwargs,
)
Docstring:     
Connect to and submit computation to a Dask cluster

The Client connects users to a Dask cluster.  It provides an asynchronous
user interface around functions and futures.  This class resembles
executors in ``concurrent.futures`` but also allows ``Future`` objects
within ``submit/map`` calls.  When a Client is instantiated it takes over
all ``dask.compute`` and ``dask.persist`` calls by default.

It is also common to create a Client without specifying the scheduler
address , like ``Client()``.  In this case the Client creates a
:class:`LocalCluster` in the background a

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Assume that we have the "chartedIn" results
import takco

tables = takco.HashBag._load('../../output/chartedIn/2-cluster/*')

print(f"Got {sum(1 for _ in tables)} tables")

# Get biggest table
table = sorted(tables, key=lambda x: -x.get('numDataRows'))[0]
takco.preview( table )

Got 61 tables


?,0,1,2,3,4,5,6,7,8,9,10,11,12,13
,Chart,_pgTitle,Position,_Variable,_pgTitle,Peak position,_Variable,_pgTitle,Peak position,_pgTitle,Position,_pgTitle,Year,Position
,Argentine Albums Chart,These Days (Bon Jovi album),3,1995,,,,,,,,,,
,Australian Albums Chart,These Days (Bon Jovi album),1,1995,,,,,,,,,,
,Austrian Albums Chart,These Days (Bon Jovi album),1,1995,,,,,,,,,,
,Belgian Albums Chart (Flanders),These Days (Bon Jovi album),3,1995,,,,,,,,,,
,Belgian Albums Chart (Wallonia),These Days (Bon Jovi album),6,1995,,,,,,,,,,


In [3]:
for t in table.get('partColAligns'):
    t['_id'] = t['tableIndex']
takco.preview( table.get('partColAligns') )

?,0,1,2,3
,_pgTitle,Position,_Variable,Chart
,These Days (Bon Jovi album),3,1995,Argentine Albums Chart
,These Days (Bon Jovi album),1,1995,Australian Albums Chart
,These Days (Bon Jovi album),1,1995,Austrian Albums Chart
,These Days (Bon Jovi album),3,1995,Belgian Albums Chart (Flanders)
,These Days (Bon Jovi album),6,1995,Belgian Albums Chart (Wallonia)
?,0,1,2,3
,_pgTitle,Peak position,_Variable,Chart
,So Fresh: The Hits of Summer 2010 + the Best of 2009,1,2009,Australian ARIA Compilations Chart
,Unsainted,86,2019,Australia ( ARIA )


In [15]:
import pandas as pd
sims = pd.read_csv('../../output/chartedIn/sims.csv', index_col=[0,1,2,3])

partColAligns = table.get('partColAligns')
ci_head = {}
for p in partColAligns:
    head = [' '.join(c.get('text','') for c in hcol) for hcol in zip(*p['tableHeaders'])]
    l,g = p['partcol_local'], p['partcol_global']
    ci_head.update({ci:head[ l[pci] ] for pci,ci in g.items()})
ci_head = pd.Series(ci_head, name='header')

partsims = sims.join(ci_head, on='ci1', how='inner').join(ci_head, how='inner', on='ci2', rsuffix='2')

partsims = partsims[partsims.header == 'Peak position']
partsims.sort_values('bodyvec', ascending=False).head(60)

headjacc   bodylsh  bodyvec         header        header2
ti1 ti2 ci1 ci2                                                           
10  5   50  24        0.0  0.000000      0.0  Peak position       _pgTitle
5   19  25  94        0.0  0.000000      0.0  Peak position           Year
    3   25  19        0.0  0.000000      0.0  Peak position          Chart
10  3   50  18        0.0  0.000000      0.0  Peak position      _Variable
5   3   25  18        0.0  0.000000      0.0  Peak position      _Variable
10  3   50  17        0.5  0.312500      0.0  Peak position       Position
5   3   25  17        0.5  0.734375      0.0  Peak position       Position
10  3   50  16        0.0  0.000000      0.0  Peak position       _pgTitle
5   3   25  16        0.0  0.000000      0.0  Peak position       _pgTitle
10  19  50  96        0.5  0.371094      0.0  Peak position       Position
5   19  25  96        0.5  0.230469      0.0  Peak position       Position
10  19  50  95        0.0  0.000000      0.0  Peak position          Chart
5   19  25  95        0.0  0.000000      0.0  Peak position          Chart
10  19  50  94        0.0  0.000000      0.0  Peak position           Year
            93        0.0  0.000000      0.0  Peak position       _pgTitle
    5   50  25        1.0  0.351562      0.0  Peak position  Peak position
5   19  25  93        0.0  0.000000      0.0  Peak position       _pgTitle
10  16  50  80        0.5  0.464844      0.0  Peak position       Position
5   16  25  80        0.5  0.261719      0.0  Peak position       Position
10  16  50  79        0.0  0.000000      0.0  Peak position          Chart
5   16  25  79        0.0  0.000000      0.0  Peak position          Chart
10  16  50  78        0.0  0.000000      0.0  Peak position       _pgTitle
5   16  25  78        0.0  0.000000      0.0  Peak position       _pgTitle
    10  25  50        1.0  0.351562      0.0  Peak position  Peak position
            49        0.0  0.000000      0.0  Peak position          Chart
            48        0.0  0.000000      0.0  Peak position       _pgTitle
10  5   50  27        0.0  0.000000      0.0  Peak position          Chart
            26        0.0  0.000000      0.0  Peak position      _Variable
    3   50  19        0.0  0.000000      0.0  Peak position          Chart

In [14]:
set(partsims.header)

{'Chart', 'Peak position', 'Position', 'Year', '_Variable', '_pgTitle'}